In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import cv2
from gait import *
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import glob

In [2]:
# 1,1,3,128,128 => 1-sample/batch_size, 1-channels, 3 - #frames, 128 - height, 128 - width
x = torch.rand((1,1,3,120,160))
y = torch.rand((1,1,1,64,64))

In [3]:
x.shape,y.shape

(torch.Size([1, 1, 3, 120, 160]), torch.Size([1, 1, 1, 64, 64]))

In [4]:
class Conv3D(nn.Module):

    '''
    Performs 3D convolution
    '''

    def __init__(self):

        super(Conv3D, self).__init__()
        self.conv1 = self._convblock(1,16,3,3,False)
        self.conv2 = self._convblock(16,32,1,3,False)
        self.conv3 = self._convblock(32,16,1,3,False)
        self.conv4 = self._convblock(16,1,1,3,True)

    def _convblock(self,in_channels,out_channels,ksized,ksize,last):
        '''
        Makes a block of layers (Conv3d,ReLU,Maxpool3d,BatchNorm3d(only if !last))
        '''

        l1 = nn.Conv3d(in_channels=in_channels,out_channels=out_channels,kernel_size=(ksized,ksize,ksize))
        l2 = nn.ReLU()
        if last:
            return nn.Sequential(l1,l2)
        else:
            l3 = nn.MaxPool3d((1, 1, 1))
            l4 = nn.BatchNorm3d(out_channels)
            return nn.Sequential(l1,l2,l3,l4)
        
        
    def forward(self,x):

        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)

        return out


In [5]:
model = Conv3D()

In [6]:
out = model(x)
out.shape

torch.Size([1, 1, 1, 112, 152])

In [7]:
class PEIData(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
    
    
    


In [8]:
from get_imfromclus import getims


In [12]:
paths_4 = getims(cluster=4)

In [13]:
paths_4.sort()

In [14]:
# for i in range(50):
#     cv2.imshow('',cv2.imread(paths_4[i]))
#     cv2.waitKey(0)
# cv2.destroyAllWindows()

In [37]:
# os.chdir('~/Downloads/')
dataset = []

angles = sorted(os.listdir(os.getcwd()+'/GaitDatasetB-silh/'+ '001' + '/bg-01'))

for angle in angles:
    images = []
    for exp in range(1,2):
        exp = f"{exp:03}" +  '/'
        # print(exp)
        for subject in sorted(os.listdir(os.getcwd()+'/GaitDatasetB-silh/'+ exp)):
            subject = subject + '/'
            # print(subject)
            dir1 = os.getcwd()+'/GaitDatasetB-silh/'+exp+subject+angle
            mask = [path for path in paths_4 if dir1 in path]
            images.append(np.asarray([cv2.imread(file) for file in mask]))
            

            # images = np.concatenate((a, b), axis=0)
    # print(len(images))
    # images = [np.concatenate((images[i-1],images[i]),axis=0) for i in range(1,len(images))]
    images = np.concatenate(images)
    dataset.append(images)

In [38]:
len(dataset)

11

In [41]:
# for im in dataset[10]:
#     cv2.imshow('',im)
#     cv2.waitKey(0)
# cv2.destroyAllWindows()